# Match Fines to Streets and Sans Citations
2/24/24

This notebook connects fines for unshoveled sidewalks to citations from the Streets and Sanitation department, based on data from two distinct FOIA responses:

|Source|Description|Filename|
|---|---|---|
|Administrative Hearings|citations from January 1, 2019 to the present, related to uncleared sidewalks following snowfall|[FOIA-H064920-011124-snow-clearance-fines.csv](../data/01-raw/FOIA-H064920-011124-snow-clearance-fines.csv)|
|Streets and Sanitation|citations from July 1, 2019 to the present, related to uncleared sidewalks following snowfall|[FOIA F067484-021424- streets and san snow clearance citations.csv](../data/01-raw/FOIA-F067484-021424-streets-and-san-snow-clearance-citations.csv)|

### questions this analysis raises
what are incident types?
* DSS_OTHER
* VACANT_LOT
* DSS_DUMPSTER
* LOT_CLEANING
* TRASH_JUNK
* WEED_CUTTING

what are the dispositions?

DISPOSITION
* Liable
* Default
* Non-Suit
* Not Liable
* Dismissed Without Prejudice

# 1. Assess Streets and Sans Citations Dataset

### takeaways about the dataset
<ul>
    <li>649 citations provided in the Streets and Sans dataset; of these:
            <ul>
                <li>597 have a unique docket number
                <li>52 have no docket assigned
            </ul>
    <li>Unique identifier is DOCKET NUMBER
    <li>all 497 of Streets and Sans records from Admin Hearings fines dataset are matched
        <li>this provides three potentially useful fields: VIOLATION CODE, INCIDENT TYPE, DISPOSITION
    </ul>

In [1]:
import pandas as pd

### read citations dataset

In [2]:
df_streets_citations = pd.read_csv("../data/01-raw/FOIA-F067484-021424-snow-citations-streets-and-san.csv")
df_streets_citations.head()

,INSPECTOR NAME,INSPECTOR BADGE,VIOLATION CODE,INCIDENT TYPE,VIOLATION TYPE,DATE OF VIOLATION,INCIDENT NUMBER,DOCKET NUMBER,DEPARTMENT,DISPOSITION,WARD,DISTRICT,BEAT,PRECINCT,ADDRESS,CENTRAL BUSINESS DIST
0,Hal Eason,1302,10-8-180; 7-28-450(a); 7-28-710,DSS_OTHER,PROPERTY,Jan-24-2024,551421,NaN,DSS,NaN,9,5.0,532.0,NaN,"12411 S MICHIGAN AVE, Chicago, IL 60628",NaN
1,RANDY MOSBY,1517,10-8-180; 7-28-260(a); 7-28-261(b); 7-28-710,DSS_OTHER,PROPERTY,Jan-23-2024,551360,NaN,DSS,NaN,16,7.0,715.0,NaN,"5757 S MARSHFIELD AVE, Chicago, IL 60636",NaN
2,DANIEL HUICOCHEA,1519,10-8-180,DSS_OTHER,PROPERTY,Jan-22-2024,551338,NaN,DSS,NaN,14,8.0,824.0,NaN,"3101 W 59TH ST, Chicago, IL 60629",NaN
3,Marchelle Collins,1514,10-8-180; 7-28-060,DSS_OTHER,PROPERTY,Jan-22-2024,551336,NaN,DSS,NaN,3,1.0,133.0,NaN,"2620 S DEARBORN ST, Chicago, IL 60616",NaN
4,Michael Caputo,1492,10-8-180,DSS_OTHER,PROPERTY,Jan-19-2024,551262,24DS01697M,DSS,Not Liable,38,16.0,1614.0,NaN,"8354 W IRVING PARK RD, Chicago, IL 60634",NaN


In [3]:
len(df_streets_citations)

649

In [4]:
len(df_streets_citations[df_streets_citations['DOCKET NUMBER'].isna()])

52

In [5]:
df_streets_citations['DOCKET NUMBER'].nunique()

597

In [6]:
# inspectors. Note that Francisco Arroyo is the inspector for the 15th ward, Donte Wilson is mostly 16th
df_streets_citations.groupby('INSPECTOR NAME').size().reset_index(name="n").sort_values('n', ascending=False)

,INSPECTOR NAME,n
14,Francisco Arroyo,217
11,Donte Winston,103
31,Nathan Wilson,58
9,Danny Murphy,42
39,Reginald Garrett,34
49,William Hohenadel,27
15,Gary Thomas,24
0,Adam Corona,21
44,Shawn Bradley,16
38,Rafael Coreas,15


In [7]:
# violation code
df_streets_citations.groupby('VIOLATION CODE').size().reset_index(name="n").sort_values('n', ascending=False).head()

,VIOLATION CODE,n
2,10-8-180,550
44,10-8-180; 7-28-750(b),11
41,10-8-180; 7-28-750(B),11
33,10-8-180; 7-28-710,10
40,10-8-180; 7-28-750(A); 7-28-750(B),5


In [8]:
# incident type
df_streets_citations.groupby('INCIDENT TYPE').size().reset_index(name="n").sort_values('n', ascending=False)

,INCIDENT TYPE,n
1,DSS_OTHER,621
4,VACANT_LOT,9
0,DSS_DUMPSTER,6
2,LOT_CLEANING,6
3,TRASH_JUNK,5
5,WEED_CUTTING,2


In [9]:
# violation type
df_streets_citations.groupby('VIOLATION TYPE').size().reset_index(name="n").sort_values('n', ascending=False).head()

,VIOLATION TYPE,n
0,PROPERTY,649


In [10]:
# department
df_streets_citations.groupby('DEPARTMENT').size().reset_index(name="n").sort_values('n', ascending=False).head()

,DEPARTMENT,n
0,DSS,649


In [11]:
# disposition
df_streets_citations.groupby('DISPOSITION').size().reset_index(name="n").sort_values('n', ascending=False).head()

,DISPOSITION,n
3,Liable,332
1,Default,189
4,Non-Suit,58
5,Not Liable,15
2,Dismissed Without Prejudice,2


# 2. Merge With Fines Dockets

### read dockets
this dataset was created in fines-03-summarize_ssw_part2

In [12]:
df_dockets = pd.read_csv("../data/04-standardized/fines_by_docket.csv")
df_dockets.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/04-standardized/fines_by_docket.csv'

In [ ]:
# review # of dockets by department
df_dockets.groupby('dept').size()

### merge datasets

In [ ]:
df_merged=pd.merge(df_dockets,df_streets_citations,left_on="docket",right_on="DOCKET NUMBER")

In [ ]:
df_merged.head()

In [ ]:
df_merged = df_merged[['docket','dept','address','lat','long','community',
                       'violation_date','total_fine','VIOLATION CODE', 'INCIDENT TYPE', 'DISPOSITION']]

In [ ]:
df_merged = df_merged.rename(columns={'VIOLATION CODE':'violation_code','INCIDENT TYPE':'incident_type','DISPOSITION':'disposition'})

In [ ]:
df_merged.head()

In [ ]:
len(df_merged)

In [ ]:
# whoa! all 497 streets and san records match!!!
df_merged['docket'].nunique()

In [ ]:
# by department
df_merged.groupby('dept').size().reset_index(name='n').sort_values(by='n',ascending=False)

In [ ]:
df_merged.dtypes

# 3. Summarize Merged Dataset

In [ ]:
# by community
df_merged.groupby('community').size().reset_index(name='n').sort_values(by='n',ascending=False).head()

# 4. Closer Assesment of Englewood

In [ ]:
df_englewood = df_merged[df_merged['community']=='ENGLEWOOD']

In [ ]:
df_englewood

In [ ]:
# incident type
df_englewood.groupby('incident_type').size().reset_index(name="n").sort_values('n', ascending=False).head()

In [ ]:
# violation code
df_englewood.groupby('violation_code').size().reset_index(name="n").sort_values('n', ascending=False).head()

In [ ]:
# disposition
df_englewood.groupby('disposition').size().reset_index(name="n").sort_values('n', ascending=False).head()

# 5. Export for Analysis

In [ ]:
df_merged.to_csv("../data/04-standardized/fines_by_docket_streets.csv")

In [ ]:
df_merged.to_csv("../data/04-standardized/fines_by_docket_streets_englewood.csv")